In [0]:
# Instalamos KERAS
!pip install -q keras

In [0]:
# Descargamos el dataset
!git clone https://github.com/agonzalezhidalgo/dataset_ger.git

Cloning into 'dataset_ger'...
remote: Enumerating objects: 51927, done.
remote: Counting objects: 100% (51927/51927), done.
remote: Compressing objects: 100% (51754/51754), done.
remote: Total 51927 (delta 172), reused 51927 (delta 172), pack-reused 0
Receiving objects: 100% (51927/51927), 341.66 MiB | 32.15 MiB/s, done.
Resolving deltas: 100% (172/172), done.
Checking out files: 100% (51886/51886), done.


In [0]:
# http://scikit-image.org/docs/stable/api/api.html
import skimage

# https://docs.python.org/3/library/csv.html
import csv

# https://matplotlib.org/api/index.html
import matplotlib

# https://matplotlib.org/api/_as_gen/matplotlib.pyplot.html#module-matplotlib.pyplot
import matplotlib.pyplot as plt

# https://docs.python.org/3/library/os.html
import os

# https://docs.scipy.org/doc/numpy/reference/
import numpy as np

# https://keras.io/models/model/
import keras

from keras import models

# Core Layers: https://keras.io/layers/core/
# Convolution Layers: https://keras.io/layers/convolutional/
from keras import layers

# https://keras.io/preprocessing/image/
from keras.preprocessing.image import ImageDataGenerator

# https://keras.io/callbacks/
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


Celda con funciones generales

In [0]:
# Devuelve las imagenes y etiquetas de una carpeta especifíca
def __read_images_labels_from_dir(directory, images, labels, shape, as_gray, get_label_from_dir):
    
    #Buscamos fotos en el directorio
    for f in os.listdir(directory):
        # Cargamos archivos con extension .ppm 
        # Obviamos el color y lo cargamos como escala de grises y normalizamos el tamaño
        if f.endswith(".ppm"):
            image = load_image(os.path.join(directory, f), shape, as_gray)
            images.append(image)
            if get_label_from_dir:
                labels.append(int(os.path.basename(directory)))
    
    return images, labels
  
# Obtenemos el índice inicial y final de una label específica
# - label: índice de la label que se quiere buscar.
# - source: lista de la relación de etiquetas.

def __get_start_end(label, source):

    start = 0
    end = 0
    
    try:        
        start = source.index(label)
        end = start + source.count(label)

    except:
        print("label doesn't exist")

    return start, end
  
# Devuelve una colección de directorios de la ruta
def __get_directories(data_dir):
    
    directories = []
    
    if os.path.exists(data_dir):
        
        #Buscamos todos los directorios de la ruta
        for d in os.listdir(data_dir):
            if os.path.isdir(os.path.join(data_dir, d)):
                directories.append(d)
                
    else:
        print("path doesn't exists")
    return directories
  
# Devuelve una colección con las imágenes y los labels de la ruta
# -datadir: path donde se encuentran la colección de imágenes.
# -shape: Dimensiones con las que se cargarán las imágenes.
# -as_gray: Indica si la imagen se cargará en escala de grises.
# -get_label_from_dir: (defecto True) específica si la categoría se lee del propio directorio
def readDataset(data_dir, shape, as_gray, get_label_from_dir = True):
    images = []
    labels = []
    directories = __get_directories(data_dir)
    
    images, labels = __read_images_labels_from_dir(data_dir, images, labels, shape, as_gray, get_label_from_dir)
    
    for d in directories:
                
        #Buscamos fotos en el directorio
        images_dir = os.path.join(data_dir, d)
        images, labels = __read_images_labels_from_dir(images_dir, images, labels, shape, as_gray, get_label_from_dir)
       
    return images, labels
  
# Devuelve una imagen
# - path: ruta de la imagen.
# - size: dimensión con la que se cargará la imagen.
# - as_gray: True para cargar la imagen en escala de grises.

def load_image(path, size, as_gray):
    aux = skimage.data.imread(path, as_gray = as_gray)
    return skimage.transform.resize(aux, size, mode='constant')
  
# Devuelve una lista con los datos del fichero csv.
# - path: ruta hasta el fichero csv.
# - delimeter: Carácter delimitador de campos

def read_csv(path, delimiter):
    file = open(path)
    title_csv = csv.reader(file, delimiter = delimiter)
    return list(title_csv)
  
# Imprime los tamaños de las colecciones
# - images: lista de imágenes precargadas.
# - labels: relación de las imágenes con las categorías a las que pertenecen.
# - np_images: np.array con las imágenes precargadas.
# - np_labels: np.array con las categorías precargadas.
# - environment: string identificativo del entorno.

def print_size_dataset(images, labels, np_images, np_labels, environment):
    print("Total images (" + environment + "): ", len(images))
    print("Total labels (" + environment + "): ", len(set(labels)))
    print("Images shape: ", np_images.shape)
    print("Labels shape: ", np_labels.shape)
    
# Imprime una matriz 32x32 con los diferentes tipos de señales que
# se van a clasificar. Muestra la primera imagen de cada categoría.
# - images: lista de imágenes precargadas.
# - labels: relación de las imágenes con las categorías a las que pertenecen.
# - titles: lista con los nombres de las categorías.

def print_summary_dataset(images, labels, titles):
    
    #Quitamos los repetidos a los labels
    unique_labels = set(labels)
    
    plt.figure(figsize=(32, 32))
    i = 1
    
    for label in unique_labels:
        
        #Obtenemos la primera imagén de cada label
        image = images[labels.index(label)]
        plt.subplot(8, 8, i)
        plt.axis('off')
        plt.title(titles[i-1][1])
        i += 1
        _ = plt.imshow(image)
        
    plt.show()

# Imprime todas las imágenes de una label especifica
# - label: índice de la label que se quiere buscar.
# - images: lista de imágenes precargadas.
# - source: lista de la relación de etiquetas.
# - titles: lista con las diferentes categorías de labels.

def print_signals(label, images, source, titles):
    
    start, end = __get_start_end(label, source)
    
    if start < end:
        rows = (end - start) / 8

        print("Signal: ", titles[label][1])
        plt.figure(figsize=(10, 10))
        i = 1

        for image in images[start:end]:
            plt.subplot(rows + 1, 8, i)
            plt.axis('off')
            i += 1
            plt.imshow(image)

        plt.show()
        
# Imprime los atributos de las imágenes de una label especifica.
# - label: índice de la label que se quiere buscar.
# - images: lista de imágenes precargadas.
# - source: lista de la relación de etiquetas.
# - titles: lista con las diferentes categorías de labels.

def print_signals_attributes(label, images, source, titles):
    
    start, end = __get_start_end(label, source)
    
    if start < end:
        print("Signal: ", titles[label][1])
        for image in images[start:end]:
            print("shape: ", image.shape, "\tmin:", image.min(), "\tmax: ", image.max())
            
# Devuelve una lista con las categorías de las imágenes de prueba leídas del fichero csv.
# - csv: fichero csv
# - class_column: número de columna que contiene las clases.
# - first_is_header: Indica si la primera fila es una cabecerá.

def get_class_id_array(csv, class_column, first_is_header = True):
    labels = []
    for row in csv:
        if not first_is_header:
            labels.append(int(row[class_column]))
        else:
            first_is_header = False
    
    return labels

In [0]:
# Mostramos información de CPU & GPU

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14237383696818439362
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 18346665061786467070
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 17489262739820913928
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14800692839
locality {
  bus_id: 1
  links {
  }
}
incarnation: 7700283072637644804
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
]


In [0]:
# Obtenemos el directorio actual como trabajo.
ROOT_PATH = os.getcwd()

# Establecemos la dimensión de las imágenes.
IMG_SHAPE = (32, 32)
print("Tamaño de las imágenes de entrada: ", IMG_SHAPE)
IMG_SHAPE_LEN = IMG_SHAPE[0] * IMG_SHAPE[1]
print("Vectorizando la entrada, sería de un tamaño: ", IMG_SHAPE_LEN)

# Obtenemos los paths de trabajo
labels_path = os.path.join(ROOT_PATH, "dataset_ger/labels.csv")
train_path = os.path.join(ROOT_PATH, "dataset_ger/train/Images")
test_info_path = os.path.join(ROOT_PATH, "dataset_ger/test/GT-final_test.csv")
test_path = os.path.join(ROOT_PATH, "dataset_ger/test/Images")

Tamaño de las imágenes de entrada:  (32, 32)
Vectorizando la entrada, sería de un tamaño:  1024


In [0]:
# Cargamos las imágenes de entrenamiento.
images_train, labels_train = readDataset(train_path, IMG_SHAPE, False)

# Convertimos las listas a array numpy de float32
np_images_train = np.asarray(images_train, dtype = np.float32)
np_labels_train = np.asarray(labels_train, dtype = np.int8)

# Recuperamos los nombres de las categorias. Los diferentes tipo de señales
# que se van a clasificar.
titles = read_csv(labels_path, ",")

# Se imprime información de los datos cargados.
print_size_dataset(images_train, labels_train, np_images_train, np_labels_train, "train")
print("Titles total: ", len(titles))

/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


Total images (train):  39209
Total labels (train):  43
Images shape:  (39209, 32, 32, 3)
Labels shape:  (39209,)
Titles total:  43


In [0]:
# Cargamos las imágenes de entrenamiento.
images_test, labels_test = readDataset(test_path, IMG_SHAPE, False, False)

test_info  = read_csv(test_info_path, ";")
labels_test = get_class_id_array(test_info, 7)

# Convertimos las listas a array numpy de float32
np_images_test = np.asarray(images_test, dtype = np.float32)
np_labels_test = np.asarray(labels_test, dtype = np.int8)

# Se imprime información de los datos cargados.
print_size_dataset(images_test, labels_test, np_images_test, np_labels_test, "test")

/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


Total images (test):  12630
Total labels (test):  43
Images shape:  (12630, 32, 32, 3)
Labels shape:  (12630,)


In [0]:
# Convertimos las labels de manera categórica
labels_categorical_train = keras.utils.to_categorical(np_labels_train)
labels_categorical_test = keras.utils.to_categorical(np_labels_test)

print("Ejemplo primera imagen de manera categórica: ", labels_categorical_train[0])

Ejemplo primera imagen de manera categórica:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [0]:
def get_keras_model():
    # IMPLEMENTACIÓN RED NEURONAL
    # En Keras la envoltura para cualquier red neuronal se crea con la clase Sequential
    model = models.Sequential()

    model.add(layers.Conv2D(32, (5, 5),
                            activation='relu', input_shape=(IMG_SHAPE[0], IMG_SHAPE[1], 3)))
    model.add(layers.MaxPooling2D(2, 2))
    model.add(layers.Dropout(0.2))

    model.add(layers.Conv2D(64, (5, 5), activation='relu'))
    model.add(layers.MaxPooling2D(2, 2))
    model.add(layers.Dropout(0.2))
    
    model.add(layers.MaxPooling2D(2, 2))
    model.add(layers.Dropout(0.2))

    model.add(layers.Flatten())
    model.add(layers.Dense(len(set(labels_train)), activation='softmax'))
    return model

In [0]:
model = get_keras_model()

# Muestra la arquitectura de nuestra red neuronal
model.summary()

# Configurando el modelo de aprendijaze:
#  · loss, función para evaluar el grado de error entre salidas calculadas
#  · optimizador, función para calcular los pesos de los parámetros a partir de los datos de entrada
#  · metricas, para monitorizar el proceso de aprendizaje de la red.
model.compile(loss="categorical_crossentropy",
             optimizer="sgd",
             metrics=['accuracy'])

# Entrenamiento del modelo
# - batch_size, indica el número de datos que se usan en cada actualización.
# - epochs, indica el número de veces que se usan todos los datos del proceso.
#model.fit(np_images_train, labels_categorical_train, batch_size=32, epochs=20)
model.fit(np_images_train, labels_categorical_train,
          batch_size = 32,
          epochs = 20,
          callbacks=[ModelCheckpoint('model_32_32_gpu_ger.h5', save_best_only = False)])

# Evaluación del modelo
test_loss, test_acc = model.evaluate(np_images_test, labels_categorical_test)
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 32)        2432      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 10, 10, 64)        51264     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
dropout_2 (Dropout)  

In [0]:
# Evaluación del modelo
test_loss, test_acc = model.evaluate(np_images_test, labels_categorical_test)
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

12630/12630 [==============================] - 1s 55us/step
Test loss: 12.99370772542221
Test accuracy: 0.03602533650275553


In [0]:
datagen = ImageDataGenerator(featurewise_center=False,
                             featurewise_std_normalization=False, 
                             width_shift_range=0.1,
                             height_shift_range=0.1,
                             zoom_range=0.2,
                             shear_range=0.1,
                             rotation_range=10.,)

datagen.fit(np_images_test)

In [0]:
model_2 = get_keras_model()

model_2.compile(loss="categorical_crossentropy",
             optimizer="sgd",
             metrics=['accuracy'])

model_2.fit_generator(datagen.flow(np_images_train, labels_categorical_train, batch_size=32),
                      steps_per_epoch = np_images_train.shape[0],
                      epochs = 20,
                      callbacks=[ModelCheckpoint('model_data_aug_32_32_gpu_ger.h5',save_best_only = False)])

Epoch 1/20
39209/39209 [==============================] - 628s 16ms/step - loss: 0.8090 - acc: 0.7696
Epoch 2/20
39209/39209 [==============================] - 634s 16ms/step - loss: 0.2264 - acc: 0.9333
Epoch 3/20
39209/39209 [==============================] - 629s 16ms/step - loss: 0.1605 - acc: 0.9527
Epoch 4/20
39209/39209 [==============================] - 628s 16ms/step - loss: 0.1300 - acc: 0.9616
Epoch 5/20
39209/39209 [==============================] - 627s 16ms/step - loss: 0.1108 - acc: 0.9672
Epoch 6/20
39209/39209 [==============================] - 634s 16ms/step - loss: 0.0980 - acc: 0.9707
Epoch 7/20
39209/39209 [==============================] - 632s 16ms/step - loss: 0.0885 - acc: 0.9736
Epoch 8/20
39209/39209 [==============================] - 628s 16ms/step - loss: 0.0814 - acc: 0.9758
Epoch 9/20
39209/39209 [==============================] - 627s 16ms/step - loss: 0.0759 - acc: 0.9776
Epoch 10/20
39209/39209 [==============================] - 627s 16ms/step - loss: 

In [0]:
# Evaluación del modelo
test_loss, test_acc = model_2.evaluate(np_images_test, labels_categorical_test)
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

12630/12630 [==============================] - 1s 71us/step
Test loss: 14.951218102908946
Test accuracy: 0.0371338083950754
